# 界面设计


In [1]:
import gradio as gr
import requests


def answer_question(message, history):
    url = "http://localhost:11434/api/generate"
    # 构建会话历史
    prompt = "你是个叫小明的小学生"
    if history is not None:
        for h in history:
            prompt += f"用户：{h[0]}\n助手：{h[1]}\n"
    prompt += f"用户：{message}\n助手："

    payload = {
        "model": "qwen2.5:0.5b",  # 请替换为您的模型名称
        "prompt": prompt,
        "stream": False,
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        data = response.json()
        output = data.get("response", "")
        history = history or []
        history.append((message, output.strip()))
        return "", history
    else:
        history = history or []
        history.append((message, "错误：无法从 Ollama 获取响应"))
        return "", history


def submit_file():
    pass

In [13]:
# 模拟用户验证函数
from numpy import source


def verify_user(username, password):
    # 这里可以替换为实际的用户验证逻辑
    if username == "1" and password == "1":
        return True
    return False


# 登录函数
def login(username, password):
    if verify_user(username, password):
        return gr.update(visible=False), gr.update(visible=True), "登陆成功!"
    else:
        return gr.update(visible=True), gr.update(visible=False), "用户名或密码错误!"


# 模拟专家验证函数
def verify_expert(username, password):
    # 这里可以替换为实际的用户验证逻辑
    if username == "2" and password == "2":
        return True
    return False


# 专家登录函数
def expert_login(username, password):
    if verify_expert(username, password):
        return gr.update(visible=False), gr.update(visible=True), "登陆成功!"
    else:
        return gr.update(visible=True), gr.update(visible=False), "用户名或密码错误!"


# 退出登录函数
def logout():
    return gr.update(visible=True), gr.update(visible=False), ""


with gr.Blocks() as demo:
    # 登录界面
    with gr.Column(visible=True) as login_interface:
        gr.Markdown("<h1 style='text-align: center;'>登录</h1>")
        username = gr.Textbox(label="用户名")
        password = gr.Textbox(label="密码", type="password")
        login_button = gr.Button("登录", scale=1)
        login_message = gr.Markdown("")
        expert_login_button = gr.Button("专家登录", scale=1)

    # 聊天界面
    with gr.Column(visible=False) as chat_interface:
        gr.Markdown("<h1 style='text-align: center;'>AI 问答界面</h1>")
        with gr.Row():
            gr.Markdown("")
            logout_button = gr.Button("退出登录", elem_id="logout_button")
        chatbot = gr.Chatbot()
        msg = gr.Textbox(show_label=False, placeholder="请输入您的消息...")
        submit = gr.Button("提交对话")
        clear = gr.Button("清除对话")

        msg.submit(answer_question, [msg, chatbot], [msg, chatbot])
        submit.click(answer_question, [msg, chatbot], [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)
        logout_button.click(
            logout, outputs=[login_interface, chat_interface, login_message]
        )
    # 专家聊天界面
    with gr.Column(visible=False) as expert_chat_interface:
        gr.Markdown("<h1 style='text-align: center;'>专家引导界面</h1>")
        with gr.Row():
            gr.Markdown("")
            logout_button = gr.Button("退出登录", elem_id="logout_button")
        chatbot = gr.Chatbot()
        with gr.Row():
            with gr.Column():
                msg = gr.Textbox(show_label=False, placeholder="请输入您的消息...")
                submit = gr.Button("提交对话")
                clear = gr.Button("清除对话")
            with gr.Column():
                file = gr.File(label="上传所需文件")
                submit_file = gr.Button("提交文件")
                clear_file = gr.Button("清除文件")

        msg.submit(answer_question, [msg, chatbot], [msg, chatbot])
        submit.click(answer_question, [msg, chatbot], [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)
        logout_button.click(
            logout, outputs=[login_interface, expert_chat_interface, login_message]
        )

    # 绑定登录按钮的点击事件
    login_button.click(
        login,
        inputs=[username, password],
        outputs=[login_interface, chat_interface, login_message],
    )
    # 绑定专家登陆事件
    expert_login_button.click(
        expert_login,
        inputs=[username, password],
        outputs=[login_interface, expert_chat_interface, login_message],
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
